In [9]:
import logging
import time

import tensorflow_datasets as tfds
import tensorflow as tf

import os
import json

import numpy as np
import matplotlib.pyplot as plt
import random

In [6]:
with open("../datasets/open-mantra-dataset/annotation.json", encoding="utf-8") as f:
    ds = json.load(f)

In [11]:
ds[0]['pages']

[{'page_index': 1,
  'image_paths': {'ja': 'images/tojime_no_siora/ja/000.jpg'},
  'frame': [{'x': 9, 'y': 84, 'w': 733, 'h': 1080}],
  'text': [{'x': 82,
    'y': 952,
    'w': 668,
    'h': 107,
    'text_ja': '綴じ眼のシオラ',
    'text_en': 'bound eye siora',
    'text_zh': '縫眼的希奧拉'},
   {'x': 515,
    'y': 1057,
    'w': 254,
    'h': 85,
    'text_ja': '朽鷹みつき',
    'text_en': 'Mitsuki Kuchitaka',
    'text_zh': '朽鷹美月'}]},
 {'page_index': 2,
  'image_paths': {'ja': 'images/tojime_no_siora/ja/001.jpg'},
  'frame': [{'x': 61, 'y': 0, 'w': 768, 'h': 220},
   {'x': 179, 'y': 242, 'w': 597, 'h': 198},
   {'x': 334, 'y': 460, 'w': 495, 'h': 709},
   {'x': 72, 'y': 470, 'w': 240, 'h': 615}],
  'text': [{'x': 172,
    'y': 194,
    'w': 126,
    'h': 229,
    'text_ja': 'だからっ',
    'text_en': "I'm telling you!!",
    'text_zh': '所以...'},
   {'x': 692,
    'y': 519,
    'w': 98,
    'h': 184,
    'text_ja': '知らないって言ってるだろっ',
    'text_en': "I don't know what you're talking about!",
    'text_zh': 

In [ ]:
from distutils.version import LooseVersion
from skimage import transform
import skimage.io

def fix_dim(img):
    if len(img.shape) == 3:
        return img
    w, h = img.shape
    ret = np.empty((w, h, 3), dtype=np.uint8)
    ret[:, :, 0] = img
    ret[:, :, 1] = img
    ret[:, :, 2] = img
    return ret

def resize(image, output_shape, order=1, mode='constant', cval=0, clip=True,
           preserve_range=False, anti_aliasing=False, anti_aliasing_sigma=None):
    """A wrapper for Scikit-Image resize().

    Scikit-Image generates warnings on every call to resize() if it doesn't
    receive the right parameters. The right parameters depend on the version
    of skimage. This solves the problem by using different parameters per
    version. And it provides a central place to control resizing defaults.
    """
    if LooseVersion(skimage.__version__) >= LooseVersion("0.14"):
        # New in 0.14: anti_aliasing. Default it to False for backward
        # compatibility with skimage 0.13.
        return transform.resize(
            image, output_shape,
            order=order, mode=mode, cval=cval, clip=clip,
            preserve_range=preserve_range, anti_aliasing=anti_aliasing,
            anti_aliasing_sigma=anti_aliasing_sigma)
    else:
        return transform.resize(
            image, output_shape,
            order=order, mode=mode, cval=cval, clip=clip,
            preserve_range=preserve_range)

def load_image(path):
    x = tf.convert_to_tensor(resize(fix_dim(skimage.io.imread(path)), (1024, 1024)), dtype = tf.dtypes.float32)
    return fix_dim(x)

In [ ]:
tokenizers = tf.saved_model.load("model/tokenizer")

In [ ]:
def data(training=True):
    random.seed(39)
    for book in ds:
        for page in book['pages']:
            for text in page['text']:
                num = random.randint(0, 19)
                if num % 20 == 0 and training: continue
                if num % 20 != 0 and (not training): continue
                ja = text['text_ja']
                en = text['text_en']
                img = load_image("../datasets/open-mantra-dataset/" + page['image_paths']['ja'])
                if len(ja) > 64 or len(en) > 64:
                    continue
                ja_encoded = tokenizers.ja.tokenize(ja).to_tensor()
                en_encoded = tokenizers.en.tokenize(en).to_tensor()